$$ \hat{G}\hat{K}\hat{G} = \sum_{i=0}^{N-1}\sum_{\sigma}\prod_{j \neq i,i+1}e^{-2g n_{j\uparrow}n_{j\downarrow}} e^{-g n_{i\sigma+1}}e^{-g n_{i+1\sigma+1}}  \frac{k}{2}\left(X_{i\sigma}X_{i+1\sigma}+Y_{i+1\sigma}Y_{i\sigma}\right) \rightarrow \sum_{i=0}^{N-1}\sum_{\sigma}\prod_{j \neq 0,1}e^{-2g n_{j\uparrow}n_{j\downarrow}} e^{-g n_{0\sigma+1}}e^{-g n_{1\sigma+1}}  \frac{k}{2}\left(Z_0 - Z_1\right)$$

\begin{equation}
\begin{split}
G_2 K G_2 =\sum_{i\sigma} \left(\prod_{j\neq i-1,i,i+1} e^{-2g n_j n_{j+1}}\right) e^{-g n_{i-1}(1+2n_{i\sigma+1})}e^{-g (n_{i+1\sigma+1}+n_{i\sigma+1})^2}e^{-g (1+2n_{i+1\sigma+1})n_{i+2}} \left( X_{i\sigma}X_{i+1\sigma} + Y_{i+1\sigma}Y_{i\sigma} \right) \rightarrow \sum_{i\sigma} \left(\prod_{j\neq N,0,1} e^{-2g n_j n_{j+1}}\right) e^{-g n_{N}(1+2n_{0\sigma+1})}e^{-g (n_{1\sigma+1}+n_{0\sigma+1})^2}e^{-g (1+2n_{1\sigma+1})n_{2}} \left( Z_0 - Z_1 \right) 
\end{split}
\end{equation}

In [389]:
import numpy as np

def find_sset(pauli):
    sset = [i for i in range(len(pauli))] #Swapped set
    fset = []
    hset = []
    N=len(pauli)
    for i in range(2):
        if pauli[i] == 'Z' or pauli[i] == 'I':
            fset.append(i)
    for i in range(2,len(pauli)):
        if pauli[i] == 'X' or pauli[i] == 'Y':
                hset.append(i)
    for i in range(len(fset)):
        sset[hset[i]] = fset[i]
        sset[fset[i]] = hset[i]
    return sset


def analyze_GPG(g,pauli,paulis,results, post_select = True):
    N = len(pauli)
    sset = find_sset(pauli)
    idx = paulis.index(pauli)
    resultd = results[idx]
    resultu = results[0]
    ru_keys = list(resultu.keys())
    rd_keys = list(resultd.keys())
    gg = 0
    for su in ru_keys:
        for sd in rd_keys:
            wu = resultu[su]
            wd = resultd[sd]
            nu_total = 0
            nd_total = 0
            gg_sr = 1
            z0 = 1
            z1 = 1
            if sd[N-1] == '1':
                z0 = -1
            if sd[N-2] == '1':
                z1 = -1
            k_sr = z0 - z1
            for i in range(len(su)):
                nu = int(su[N-1-sset[i] ])
                nd = int(sd[N-1-i])
                nu_total += nu
                nd_total += nd
                if i == 0 or i == 1:
                    gg_sr = gg_sr*np.exp(-g*nu)
                else:
                    gg_sr = gg_sr*np.exp(-2*g*nu*nd) 
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*k_sr
            if post_select == True:
                if nu_total == 2 and nd_total == 2:
                    gg = gg + wu*wd*gg_sr*k_sr
    return gg



def analyze_GPG2(g,pauli,paulis,results, post_select = True):
    N = len(pauli)
    sset = find_sset(pauli)
    idx = paulis.index(pauli)
    resultd = results[idx]
    resultu = results[0]
    ru_keys = list(resultu.keys())
    rd_keys = list(resultd.keys())
    gg = 0
    for su in ru_keys:
        for sd in rd_keys:
            wu = resultu[su]
            wd = resultd[sd]
            nu_total = 0
            nd_total = 0
            gg_sr = 1
            z0 = 1
            z1 = 1
            if sd[N-1] == '1':
                z0 = -1
            if sd[N-2] == '1':
                z1 = -1
            k_sr = z0 - z1
            N  = len(su)
            for i in range(N):
                ui = N-1-sset[i]
                di = N-1-i
                uj = np.mod(N-2-sset[i],N)
                dj = np.mod(N-2-i,N)
                nui = int(su[ui])
                ndi = int(sd[di])
                nuj = int(su[uj])
                ndj = int(sd[dj])
                ni = nui + ndi
                nj = nuj + ndj
                nu_total += nui
                nd_total += ndi
                if i == N-1:
                    gg_sr = gg_sr*np.exp(-g*ni*(1+2*nuj))
                elif i == 0:
                    gg_sr = gg_sr*np.exp(-g*(1+2*nui)*(1+2*nuj))
                elif i == 1:
                    gg_sr = gg_sr*np.exp(-g*(1+2*nui)*nj)
                else:
                    gg_sr = gg_sr*np.exp(-2*g*ni*nj) 
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*k_sr
            if post_select == True:
                if nu_total == 2 and nd_total == 2:
                    gg = gg + wu*wd*gg_sr*k_sr
    return gg



def n_total(state):
    n_total = 0
    for i in range(len(state)):
        n_total += int(state[i])
    return n_total

#Works
def analyze_GG2(g,nbrs,result, post_select = True):
    r_keys = list(result.keys())
    gg = 0
    for su in r_keys:
        for sd in r_keys:
            wu = result[su]
            wd = result[sd]
            gg_sr = 1
            N = len(su)
            for pair in nbrs:
                i = N-1-pair[0]
                j = N-1-pair[1]
                nui = int(su[i])
                ndi = int(sd[i])
                ni = nui + ndi
                nuj = int(su[j])
                ndj = int(sd[j])
                nj = nuj + ndj
                gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr
    return gg

# Returns the expectation value of <G2(g)D(d)G2(g)> using the result measured in the z-basis 
def analyze_GDG2(g,d,nbrs,result, post_select = True):
    r_keys = list(result.keys())
    gg = 0
    for su in r_keys:
        for sd in r_keys:
            wu = result[su]
            wd = result[sd]
            gg_sr = 1
            d_sr = 0
            for i in range(len(su)):
                nu = int(su[i])
                nd = int(sd[i])
                d_sr = d_sr + d*nu*nd
            N = len(su)
            for pair in nbrs:
                i = N-1-pair[0]
                j = N-1-pair[1]
                nui = int(su[i])
                ndi = int(sd[i])
                ni = nui + ndi
                nuj = int(su[j])
                ndj = int(sd[j])
                nj = nuj + ndj
                gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*d_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr*d_sr
    return gg

# Returns the expectation value of G2(g)M(m)G2(g) using the result measured in the z-basis 
def analyze_GMG2(g,m,nbrs,result, post_select = True):
    r_keys = list(result.keys())
    gg = 0
    for su in r_keys:
        for sd in r_keys:
            wu = result[su]
            wd = result[sd]
            gg_sr = 1
            m_sr = 0
            for i in range(len(su)):
                nu = int(su[i])
                nd = int(sd[i])
                m_sr = m_sr + m*nu + m*nd
            N = len(su)
            for pair in nbrs:
                i = N-1-pair[0]
                j = N-1-pair[1]
                nui = int(su[i])
                ndi = int(sd[i])
                ni = nui + ndi
                nuj = int(su[j])
                ndj = int(sd[j])
                nj = nuj + ndj
                gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*m_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr*m_sr
    return gg





def analyze_GG2b(g,nbrs,pauli,paulis,results, post_select = True):
    N = len(pauli)
    sset = find_sset(pauli)
    idx = paulis.index(pauli)
    resultd = results[idx]
    resultu = results[0]
    ru_keys = list(resultu.keys())
    rd_keys = list(resultd.keys())
    gg = 0
    for su in ru_keys:
        for sd in rd_keys:
            wu = resultu[su]
            wd = resultd[sd]
            gg_sr = 1
            for pair in nbrs:
                ui = N-1-pair[0]
                di = N-1-sset[pair[0]]
                uj = N-1-pair[1]
                dj = N-1-sset[pair[1]]
                nui = int(su[ui])
                ndi = int(sd[di])
                ni = nui + ndi
                nuj = int(su[uj])
                ndj = int(sd[dj])
                nj = nuj + ndj
                if di == 0 and dj == 1:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2) 
                elif dj == 0 and di == 1:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2)
                elif di == 0 or di == 1:
                    gg_sr = gg_sr*np.exp(-g*(1+2*nui)*nj)
                elif dj == 0 or dj == 1:
                    gg_sr = gg_sr*np.exp(-g*ni*(1+2*nuj))
                else:
                    gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr
    return gg

def analyze_GPG2(g,nbrs,pauli,paulis,results, post_select = True):
    N = len(pauli)
    sset = find_sset(pauli)
    idx = paulis.index(pauli)
    resultd = results[idx]
    resultu = results[0]
    ru_keys = list(resultu.keys())
    rd_keys = list(resultd.keys())
    gg = 0
    for su in ru_keys:
        for sd in rd_keys:
            wu = resultu[su]
            wd = resultd[sd]
            gg_sr = 1
            z0 = 1
            z1 = 1
            if sd[N-1] == '1':
                z0 = -1
            if sd[N-2] == '1':
                z1 = -1
            k_sr = z0 - z1
            N  = len(su)
            for pair in nbrs:
                ui = N-1-pair[0]
                di = N-1-sset[pair[0]]
                uj = N-1-pair[1]
                dj = N-1-sset[pair[1]]
                nui = int(su[ui])
                ndi = int(sd[di])
                ni = nui + ndi
                nuj = int(su[uj])
                ndj = int(sd[dj])
                nj = nuj + ndj
                if di == N-1 and dj == N-2:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2) 
                elif dj == N-1 and di == N-2:
                    gg_sr = gg_sr*np.exp(-g*(nui+nuj)**2)
                elif di == N-1 or di == N-2:
                    gg_sr = gg_sr*np.exp(-g*(1+2*nui)*nj)
                elif dj == N-1 or dj == N-2:
                    gg_sr = gg_sr*np.exp(-g*ni*(1+2*nuj))
                else:
                    gg_sr = gg_sr*np.exp(-2*g*ni*nj)
            if post_select == False:    
                gg = gg + wu*wd*gg_sr*k_sr
            if post_select == True:
                if n_total(su) == 2 and n_total(sd) == 2:
                    gg = gg + wu*wd*gg_sr*k_sr 
    return gg

In [390]:
from Define_System import system
from Define_Slater_Circuit import slater_circ
from Define_Measurment_Circuits import create_circs
from Define_Quantum_Device import Quantum_Device
from Define_Analyzers import analyze_energy, post_select
from Define_Lancozos import Lancozos
systemQ = system('square',4) 
Fd = systemQ.Fld()
paulis = systemQ.pauli_strings()
circs = create_circs(Fd,paulis)
QD = Quantum_Device(backend = 'ibm_auckland', layout = [0,1,2,3,5,8,11,14])

results = QD.get_results(circs ,method = "matrix")
results = QD.chop_results(results)

In [391]:
from Define_Paulis import bkt,Mdot,X,Y,X,I

g = 0.7
k = 1.4
g2 = 0.6
d = 0.4
m = 1.3
psi_spin = systemQ.psi_spin([2,3])

In [392]:
nbrs = systemQ.neighbors()


G2 = systemQ.G2(g2)
D = systemQ.D(d)
M = systemQ.M(m)
GPG2 = Mdot([psi_spin,G2,X(2,8),X(3,8),G2,psi_spin]) + Mdot([psi_spin,G2,Y(2,8),Y(3,8),G2,psi_spin])

print( Mdot([psi_spin,G2,G2,psi_spin]) - analyze_GG2(g2,nbrs,results[0]) )
print( Mdot([psi_spin,G2,D,G2,psi_spin]) - analyze_GDG2(g2,d,nbrs,results[0]) )
print( Mdot([psi_spin,G2,M,G2,psi_spin]) - analyze_GMG2(g2,m,nbrs,results[0]) )
print( GPG2 - analyze_GPG2(g2,nbrs,paulis[1],paulis,results) )

(7.771561172376096e-16+0j)
(3.608224830031759e-16+0j)
(2.886579864025407e-15+0j)
(4.683753385137379e-17+0j)
